**Project 1 Phase 2**<br>
<br>
<table style="width:100%">
  <tr>
    <th style = "text-align: left">#</th>
    <th style = "text-align: left">Name</th>
    <th style = "text-align: left">Lastname</th>
    <th style = "text-align: left">Matr Number</th>
  </tr>
  <tr>
    <td style = "text-align: left">1</td>
    <td style = "text-align: left">Christian</td>
    <td style = "text-align: left">Peinthor</td>
    <td style = "text-align: left">11815592</td>
  </tr>
  <tr>
    <td style = "text-align: left">2</td>
    <td style = "text-align: left">Michael</td>
    <td style = "text-align: left">Weikl</td>
    <td style = "text-align: left">1154652</td>
  </tr>

  
</table>
<br>

# Table of contents
* [Problemdefinition](#Problemdefinition)
* [Basic functions](#Basic-functions)
    * [Stopping Criterion](#Stopping-Criterion)
    * [Backtracking](#Backtracking)
    * [Wolfe Condition](#Wolfe-Condition)
    * [Derivation (Taylor-Theorem)](Derivation-(Taylor-Theorem))
* [Implementation of the algorithms](#Implementation-of-the-algorithms)
    * [Steepest descent](#Steepest-descent)
    * [Newthon method](#Newthon-method)
    * [Fetcher Reeves](#Fetcher-Reeves)
    * [SR1](#SR1)
* [Executions](#Executions)


In [1]:
import numpy as np
import scipy 
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from numpy.linalg import *

# Problems to solve

Problem class to set up problems for the algorithms to be solved.

In [2]:
class Problem():

  def __init__(self):

    self.f = None
    self.grad_f = None
    self.hessian = None
    self.min_x = None

  def himmelblau(self):

    def f(x):
      return (x[0] ** 2 + x[1] - 11) ** 2 + (x[0] + x[1] ** 2 - 7) ** 2
    
    def grad_f(x):
      return np.array([4*x[0]*(x[0]**2 + x[1] - 11)+2*(x[0] + x[1]**2 - 7),
                       4*x[1]*(x[1]**2 + x[0] - 7)+2*(x[1] + x[0]**2 - 11)])
    
    def hessian(x):
      return np.array([[12*x[0]**2 + 4*x[1] - 42, 4*(x[1] + x[0])],
                       [4*(x[1] + x[0]), 12*x[1]**2 + 4*x[0] - 26]])
      
    self.f = f
    self.grad_f = grad_f
    self.hessian = hessian
    self.min_x = np.array([[3,2], [-2.805118, 3.131312], [-3.779310, -3.283186], [3.584428, -1.848126]])

  def poly_1(self):

    def f(x):
      return ((x - 7)**2 * (x - 3)**2) / 4
    
    def grad_f(x):
      return (x - 7) * (x - 5) * (x - 3)
    
    def hessian(x):
      return 3 * x**2 - 30 * x + 71

    self.f = f
    self.grad_f = grad_f
    self.hessian = hessian
    self.min_x = np.array([[3],[5],[7]])

  def rosenbrock(self):

    def f(x):
      return 100*(x[1] - x[0]**2)**2 + (1 - x[0])**2

    def grad_f(x):
      return np.array([-400*x[0]*(x[1] - x[0]**2) - 2*(1 - x[0]),
           200*(x[1] - x[0]**2)])

    def hessian(x):
      return np.array([[-400*(x[1] - 3*x[0]**2) + 2, -400*x[0]],
           [-400*x[0], 200]])
      
    self.f = f
    self.grad_f = grad_f
    self.hessian = hessian
    self.min_x = np.array([1,1])

  #this is the function below rosenbrock defined in the project description
  def func_2(self):

    def f(x):
      return 150*(x[0] * x[1])**2 + (0.5 * x[0] + 2 * x[1] - 2)**2
    
    def grad_f(x):
      return np.array([x[0] * (300 * x[1]**2 + 0.5) + 2 * x[1] - 2, 300*x[0]**2 * x[1] + 2 * x[0] + 8 * x[1] - 8])

    def hessian(x):
      return np.array([[300 * x[1]**2 + 0.5, 600 * x[0]*x[1] + 2],[600 * x[0]*x[1] + 2, 300 * x[0]**2 + 8]])

    self.f = f
    self.grad_f = grad_f
    self.hessian = hessian
    #self calculated minima
    self.min_x = np.array([[0,1],[4,0]])

# Basic functions

## Stopping Criterion

This implements a stopping criterion for when the gradient of f at xk is small relative to the gradient of f at x0
Also a maxmimum number of iterations is implemented.

In [3]:
def stop_crit(grad_f, xk, x0, i, tol=1e-8, max_iter=5000):
  if i > max_iter: 
    return True
  elif norm(grad_f(xk)) <= tol * norm(grad_f(x0)):
    return True
  return False

## Backtracking

Backtracking linesearch to find suitable step length as described in the book. steepest_descent() implements the line search steepest descent method.

In [4]:
def backtracking_alpha(f, grad_f, xk, pk, alpha0=1, rho=0.95, c=1e-4):

  alpha = alpha0

  while not f(xk + alpha * pk) <= (f(xk) + c * alpha * grad_f(xk).T @ pk):
    alpha *= rho
  
  return alpha

## Wolfe Condition

alpha_wolfe() returns a step length satisfiying the weak wolfe conditions using a bisection approach as described in https://sites.math.washington.edu/~burke/crs/408/notes/nlp/line.pdf. newton_method() implements the line search Newton method solving the equation H @ pk = -grad instead of computing the inverse of H.

In [5]:
def alpha_wolfe(f, grad_f, xk, pk, c1=1e-4, c2=0.9):

  alpha = 0
  beta = np.Inf
  t = 1

  while True:

    if f(xk + t * pk) > (f(xk) + c1 * t * (pk @ grad_f(xk))):
      beta = t
      t = 0.5 * (alpha + beta)
    elif (-pk @ grad_f(xk + t * pk)) > (-c2 * pk @ grad_f(xk)):
      alpha = t
      t = (2 * alpha if beta == np.Inf else 0.5 * (alpha + beta))
    else:
      return t

## Derivation (Taylor Theorem)

This implements the forward difference approach from the book for gradient and hessian calculation without explicit function knowledge.

In [6]:
def e_i(size, index):
  arr = np.zeros(size)
  arr[index] = 1.0
  return arr

#these return the function that computes the gradient, which can then in turn be called to compute the gradient
def approx_grad(f, e=1.1e-8):
  def grad_f(x):
    if x.size == 1:
      return (f(x + e) - f(x)) / e
    return np.array([(f(x + e * e_i(x.size, i)) - f(x)) / e for i in range(x.size)])
  return grad_f

def approx_hessian(f, e=1.1e-8):
  def hessian_f(x):
    if x.size == 1:
      return (f(x + 2*e) - 2*f(x + e) + f(x)) / e**2
    return np.array([[(f(x + e * e_i(x.size, i) + e * e_i(x.size, j)) - f(
                      x + e * e_i(x.size, i)) - f(x + e * e_i(x.size, j)) + f(
                      x)) / e**2 for j in range(x.size)] for i in range(x.size)])
  return hessian_f

# Implementation of the algorithms

## Steepest descent

In [7]:
def steepest_descent(x0, f, grad_f=None):

  conv_tol = 1e-8
  if grad_f == None:
    grad_f = approx_grad(f)
  i = 0
  xk = x0

  while not stop_crit(grad_f, xk, x0, i, tol=conv_tol):
    
    pk = -grad_f(xk)
    xk = xk + backtracking_alpha(f, grad_f, xk, pk) * pk
    i+=1
    
  print(f"\n Search terminated after iteration {i} with result: {xk}")
  return xk

## Newton method

In [8]:
def newton_method(x0, f, grad_f=None, hessian_f=None):
  
  conv_tol = 1e-8
  if grad_f == None:
    grad_f = approx_grad(f)
    hessian_f = approx_hessian(f)
  i = 0
  x = x0
  
  while not stop_crit(grad_f, x, x0, i, tol=conv_tol):
    
    pk = np.array([- (1 / hessian_f(x)) @ grad_f(x)]) if x.size == 1 else solve(hessian_f(x), -grad_f(x))
    x = x + alpha_wolfe(f, grad_f, x, pk) * pk
    i += 1
  
  print(f"\n Search terminated after iteration {i} with result: {x}")
  return x

## Fetcher Reeves

Implementation of the Fletcher Reeves nonlinear conjugate gradient method.

In [9]:
def FR(x0, f, grad_f=None):

  conv_tol = 1e-8
  if grad_f == None:
    grad_f = approx_grad(f)
  i = 0
  xk = x0
  pk = -grad_f(xk)

  while not stop_crit(grad_f, xk, x0, i, tol=conv_tol):

    xk1 = xk + backtracking_alpha(f, grad_f, xk, pk) * pk
    beta = (grad_f(xk1) @ grad_f(xk1)) / (grad_f(xk) @ grad_f(xk))
    pk = -grad_f(xk1) + beta * pk
    xk = xk1
    i += 1
 
  print(f"\n Search terminated after iteration {i} with result: {xk}")
  return xk

## SR1

SR1() implements the line search quasi newton method utilizing SR1 updating for inverse Hessian approximation and a method for stabilizing that resets H as a multiple of I inspired by the method deployed in https://www.sciencedirect.com/science/article/pii/S0898122111004202?via%3Dihub.

In [10]:
def SR1(x0 : np.array, f, grad_f, r=1e-8):

  conv_tol = 1e-8
  i = 0
  H = np.identity(x0.size)
  x = x0

  while not stop_crit(grad_f, x, x0, i, tol=conv_tol):
  
    pk = - H @ grad_f(x)
    x_1 = x + alpha_wolfe(f, grad_f, x, pk) * pk
    sk = x_1 - x
    yk = grad_f(x_1) - grad_f(x)
    rhok = 1 / (yk.T @ sk)
    
    if ((sk @ yk - yk @ H @ yk) < 0) or (abs(yk @ (sk - H @ yk)) < r * np.linalg.norm(yk) * np.linalg.norm(sk - H @ yk)) or (
        norm(H, np.inf) > 1e10):
      mu = (sk @ sk) / (yk @ sk) - ((sk @ sk)**2 / (yk @ sk)**2 - (sk @ sk) / (yk @ yk))**0.5
      H_1 = mu * np.identity(x0.size)
      H = H_1
    else:
      H_1 = H + np.outer((sk - H @ yk), (sk - H @ yk)) / ((sk - H @ yk) @ yk)
      H = H_1
    x = x_1
    i += 1

  print(f"\n Search terminated after iteration {i} with result: {x}")

  return x

# Executions

In [11]:
#left this in to show how the problem class works etc.
prob = Problem()
prob.himmelblau()
print(f"\nProblem himmelblau: \n")
print("\nAlgorithm output exact gradient: ")
x_ = steepest_descent(np.array([0,0]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[2])}")
print("\nAlgorithm output approximated gradient: ")
x_ = steepest_descent(np.array([0,0]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(grad(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[2])}")

prob = Problem()
prob.poly_1()
print(f"\nProblem poly_1: \n")
print("\nAlgorithm output exact gradient: ")
x_ = steepest_descent(np.array([1]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[2])}")
print("\nAlgorithm output approximated gradient: ")
x_ = steepest_descent(np.array([1]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(grad(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[2])}")


Problem himmelblau: 


Algorithm output exact gradient: 

 Search terminated after iteration 2816 with result: [-3.77931026 -3.28318599]

actual minima: [[ 3.        2.      ]
 [-2.805118  3.131312]
 [-3.77931  -3.283186]
 [ 3.584428 -1.848126]]

Last gradient norm: 2.6048138970693457e-07

Difference to real solution: 2.5521830106979e-07

Algorithm output approximated gradient: 

 Search terminated after iteration 1922 with result: [-3.77931026 -3.283186  ]

actual minima: [[ 3.        2.      ]
 [-2.805118  3.131312]
 [-3.77931  -3.283186]
 [ 3.584428 -1.848126]]

Last gradient norm: 2.0923755687806018e-07

Difference to real solution: 2.6212522690478664e-07

Problem poly_1: 


Algorithm output exact gradient: 

 Search terminated after iteration 221 with result: [7.00000006]

actual minima: [[3]
 [5]
 [7]]

Last gradient norm: 4.627252097937028e-07

Difference to real solution: 5.784064871505734e-08

Algorithm output approximated gradient: 

 Search terminated after iteration 210 wi

For the newton method I first struggled with solving the himmelblau function, but after some tinkering with the starting point, it worked.
Also choosing 0s as starting points seems to not work here at all. In the end I reached VERY fast convergence for all problems though.

In [12]:
prob = Problem()
prob.himmelblau()
print(f"\nProblem himmelblau: \n")
print("\nAlgorithm output: ")
x_ = newton_method(np.array([2.5,1.5]), prob.f, prob.grad_f, prob.hessian)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[0])}")




Problem himmelblau: 


Algorithm output: 

 Search terminated after iteration 6 with result: [3. 2.]

actual minima: [[ 3.        2.      ]
 [-2.805118  3.131312]
 [-3.77931  -3.283186]
 [ 3.584428 -1.848126]]

Last gradient norm: 4.39446412658135e-14

Difference to real solution: 1.4043333874306805e-15


Implementation of the Fletcher Reeves nonlinear conjugate gradient method.

In [13]:
prob = Problem()
prob.himmelblau()
print(f"\nProblem himmelblau: \n")
print("\nAlgorithm output: ")
x_ = FR(np.array([0,0]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[3])}")


Problem himmelblau: 


Algorithm output: 

 Search terminated after iteration 502 with result: [ 3.58442834 -1.84812652]

actual minima: [[ 3.        2.      ]
 [-2.805118  3.131312]
 [-3.77931  -3.283186]
 [ 3.584428 -1.848126]]

Last gradient norm: 2.5406398925923756e-07

Difference to real solution: 6.237319089753625e-07


In [14]:
prob = Problem()
prob.himmelblau()
print(f"\nProblem himmelblau: \n")
print("\nAlgorithm output: ")
x_ = SR1(np.array([0,0]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[0])}")



Problem himmelblau: 


Algorithm output: 

 Search terminated after iteration 11 with result: [3. 2.]

actual minima: [[ 3.        2.      ]
 [-2.805118  3.131312]
 [-3.77931  -3.283186]
 [ 3.584428 -1.848126]]

Last gradient norm: 1.3802086492309621e-07

Difference to real solution: 4.5270257313018665e-09


**Rosenbrock [1.2,1.2] starting point**

In [15]:
prob = Problem()
prob.rosenbrock()
print(f"\nProblem rosenbrock steepest descent [1.2,1.2]:")
print("\nAlgorithm output: ")
x_ = steepest_descent(np.array([1.2,1.2]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")
print(f"\nProblem rosenbrock steepest descent [1.2,1.2] approximated gradients:")
print("\nAlgorithm output: ")
x_ = steepest_descent(np.array([1.2,1.2]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(grad(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")

prob = Problem()
prob.rosenbrock()
print(f"\nProblem rosenbrock newton method [1.2,1.2]:")
print("\nAlgorithm output: ")
x_ = newton_method(np.array([1.2,1.2]), prob.f, prob.grad_f, prob.hessian)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")
print(f"\nProblem rosenbrock newton method [1.2,1.2] approximated gradients:")
print("\nAlgorithm output: ")
x_ = newton_method(np.array([1.2,1.2]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(grad(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")

prob = Problem()
prob.rosenbrock()
print(f"\nProblem rosenbrock FR [1.2,1.2]:")
print("\nAlgorithm output: ")
x_ = FR(np.array([1.2,1.2]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")
print(f"\nProblem rosenbrock FR [1.2,1.2] approximated gradients:")
print("\nAlgorithm output: ")
x_ = FR(np.array([1.2,1.2]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(grad(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")

prob = Problem()
prob.rosenbrock()
print(f"\nProblem rosenbrock:")
print("\nAlgorithm output SR1 [1.2,1.2]: ")
x_ = SR1(np.array([1.2,1.2]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")
print(f"\nProblem rosenbrock SR1 [1.2,1.2] approximated gradients:")
print("\nAlgorithm output: ")
x_ = SR1(np.array([1.2,1.2]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(grad(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")


Problem rosenbrock steepest descent [1.2,1.2]:

Algorithm output: 

 Search terminated after iteration 5001 with result: [1.00225488 1.00455996]

actual minima: [1 1]

Last gradient norm: 0.016306835034657012

Difference to real solution: 0.005087017651117713

Problem rosenbrock steepest descent [1.2,1.2] approximated gradients:

Algorithm output: 

 Search terminated after iteration 5001 with result: [1.00225053 1.00455368]

actual minima: [1 1]

Last gradient norm: 0.017387939189828356

Difference to real solution: 0.0050794602350955455

Problem rosenbrock newton method [1.2,1.2]:

Algorithm output: 

 Search terminated after iteration 8 with result: [1. 1.]

actual minima: [1 1]

Last gradient norm: 1.4360392201267428e-11

Difference to real solution: 1.7227828323731315e-13

Problem rosenbrock newton method [1.2,1.2] approximated gradients:

Algorithm output: 

 Search terminated after iteration 7 with result: [0.99999669 0.99999338]

actual minima: [1 1]

Last gradient norm: 1.779

TypeError: SR1() missing 1 required positional argument: 'grad_f'

**Rosenbrock [-1.2,1] starting point**

In [ ]:
prob = Problem()
prob.rosenbrock()
print(f"\nProblem rosenbrock steepest descent [-1.2,1]:")
print("\nAlgorithm output: ")
x_ = steepest_descent(np.array([-1.2,1]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")
print(f"\nProblem rosenbrock steepest descent [-1.2,1] approximated gradients:")
print("\nAlgorithm output: ")
x_ = steepest_descent(np.array([-1.2,1]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(grad(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")

prob = Problem()
prob.rosenbrock()
print(f"\nProblem rosenbrock newton method [-1.2,1]:")
print("\nAlgorithm output: ")
x_ = newton_method(np.array([-1.2,1]), prob.f, prob.grad_f, prob.hessian)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")
print(f"\nProblem rosenbrock newton method [-1.2,1] approximated gradients:")
print("\nAlgorithm output: ")
x_ = newton_method(np.array([-1.2,1]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(grad(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")

prob = Problem()
prob.rosenbrock()
print(f"\nProblem rosenbrock FR [-1.2,1]:")
print("\nAlgorithm output: ")
x_ = FR(np.array([-1.2,1]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")
print(f"\nProblem rosenbrock FR [-1.2,1] approximated gradients:")
print("\nAlgorithm output: ")
x_ = FR(np.array([-1.2,1]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(grad(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")

prob = Problem()
prob.rosenbrock()
print(f"\nProblem rosenbrock:")
print("\nAlgorithm output SR1 [-1.2,1]: ")
x_ = SR1(np.array([-1.2,1]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")
print(f"\nProblem rosenbrock SR1 [-1.2,1] approximated gradients:")
#for some reason SR1 with this starting point and approximated gradients seems to be running forever
"""print("\nAlgorithm output: ")
x_ = SR1(np.array([-1.2,1]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(grad(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")"""

**Rosenbrock [0,1] starting point**

In [ ]:
prob = Problem()
prob.rosenbrock()
print(f"\nProblem rosenbrock steepest descent [0.0,1.0]:")
print("\nAlgorithm output: ")
x_ = steepest_descent(np.array([0.0,1.0]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")
print(f"\nProblem rosenbrock steepest descent [0.0,1.0] approximated gradients:")
print("\nAlgorithm output: ")
x_ = steepest_descent(np.array([0.0,1.0]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(grad(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")

prob = Problem()
prob.rosenbrock()
print(f"\nProblem rosenbrock newton method [0.0,1.0]:")
print("\nAlgorithm output: ")
x_ = newton_method(np.array([0.0,1.0]), prob.f, prob.grad_f, prob.hessian)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")
print(f"\nProblem rosenbrock newton method [0.0,1.0] approximated gradients:")
print("\nAlgorithm output: ")
x_ = newton_method(np.array([0.0,1.0]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(grad(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")

prob = Problem()
prob.rosenbrock()
print(f"\nProblem rosenbrock FR [0.0,1.0]:")
print("\nAlgorithm output: ")
x_ = FR(np.array([0.0,1.0]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")
print(f"\nProblem rosenbrock FR [0.0,1.0] approximated gradients:")
print("\nAlgorithm output: ")
x_ = FR(np.array([0.0,1.0]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(grad(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")

prob = Problem()
prob.rosenbrock()
print(f"\nProblem rosenbrock:")
print("\nAlgorithm output SR1 [0.0,1.0]: ")
x_ = SR1(np.array([0.0,1.0]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")
print(f"\nProblem rosenbrock SR1 [0.0,1.0] approximated gradients:")
print("\nAlgorithm output: ")
x_ = SR1(np.array([0.0,1.0]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(grad(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")

**Rosenbrock [-1,0] starting point**

In [ ]:
prob = Problem()
prob.rosenbrock()
print(f"\nProblem rosenbrock steepest descent [-1.0,0.0]:")
print("\nAlgorithm output: ")
x_ = steepest_descent(np.array([-1.0,0.0]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")
print(f"\nProblem rosenbrock steepest descent [-1.0,0.0] approximated gradients:")
print("\nAlgorithm output: ")
x_ = steepest_descent(np.array([-1.0,0.0]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(grad(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")

prob = Problem()
prob.rosenbrock()
print(f"\nProblem rosenbrock newton method [-1.0,0.0]:")
print("\nAlgorithm output: ")
x_ = newton_method(np.array([-1.0,0.0]), prob.f, prob.grad_f, prob.hessian)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")
print(f"\nProblem rosenbrock newton method [-1.0,0.0] approximated gradients:")
print("\nAlgorithm output: ")
x_ = newton_method(np.array([-1.0,0.0]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(grad(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")

prob = Problem()
prob.rosenbrock()
print(f"\nProblem rosenbrock FR [-1.0,0.0]:")
print("\nAlgorithm output: ")
x_ = FR(np.array([-1.0,0.0]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")
print(f"\nProblem rosenbrock FR [-1.0,0.0] approximated gradients:")
print("\nAlgorithm output: ")
x_ = FR(np.array([-1.0,0.0]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(grad(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")

prob = Problem()
prob.rosenbrock()
print(f"\nProblem rosenbrock:")
print("\nAlgorithm output SR1 [-1.0,0.0]: ")
x_ = SR1(np.array([-1.0,0.0]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")
print(f"\nProblem rosenbrock SR1 [-1.0,0.0] approximated gradients:")
print("\nAlgorithm output: ")
x_ = SR1(np.array([-1.0,0.0]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(grad(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")

**Rosenbrock [0,-1] starting point**

In [ ]:
prob = Problem()
prob.rosenbrock()
print(f"\nProblem rosenbrock steepest descent [0.0,-1.0]:")
print("\nAlgorithm output: ")
x_ = steepest_descent(np.array([0.0,-1.0]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")
print(f"\nProblem rosenbrock steepest descent [0.0,-1.0] approximated gradients:")
print("\nAlgorithm output: ")
x_ = steepest_descent(np.array([0.0,-1.0]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(grad(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")

prob = Problem()
prob.rosenbrock()
print(f"\nProblem rosenbrock newton method [0.0,-1.0]:")
print("\nAlgorithm output: ")
x_ = newton_method(np.array([0.0,-1.0]), prob.f, prob.grad_f, prob.hessian)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")
print(f"\nProblem rosenbrock newton method [0.0,-1.0] approximated gradients:")
print("\nAlgorithm output: ")
x_ = newton_method(np.array([0.0,-1.0]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(grad(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")

prob = Problem()
prob.rosenbrock()
print(f"\nProblem rosenbrock FR [0.0,-1.0]:")
print("\nAlgorithm output: ")
x_ = FR(np.array([0.0,-1.0]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")
print(f"\nProblem rosenbrock FR [0.0,-1.0] approximated gradients:")
print("\nAlgorithm output: ")
x_ = FR(np.array([0.0,-1.0]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(grad(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")

prob = Problem()
prob.rosenbrock()
print(f"\nProblem rosenbrock:")
print("\nAlgorithm output SR1 [0.0,-1.0]: ")
x_ = SR1(np.array([0.0,-1.0]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")
print(f"\nProblem rosenbrock SR1 [0.0,-1.0] approximated gradients:")
print("\nAlgorithm output: ")
x_ = SR1(np.array([0.0,-1.0]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(grad(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x)}")

**Function 2 [-0.2,1.2] starting point**

In [ ]:
prob = Problem()
prob.func_2()
print(f"\nProblem func_2 [-0.2,1.2] steepest descent:")
print("\nAlgorithm output: ")
x_ = steepest_descent(np.array([-0.2,1.2]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[1])}")
print(f"\nProblem func_2 [-0.2,1.2] steepest descent approximate gradient:")
print("\nAlgorithm output: ")
x_ = steepest_descent(np.array([-0.2,1.2]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[1])}")


prob = Problem()
prob.func_2()
print(f"\nProblem func_2 [-0.2,1.2] newton method:")
print("\nAlgorithm output: ")
x_ = newton_method(np.array([-0.2,1.2]), prob.f, prob.grad_f, prob.hessian)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[1])}")
print(f"\nProblem func_2 [-0.2,1.2] newton method approximate gradient:")
print("\nAlgorithm output: ")
x_ = newton_method(np.array([-0.2,1.2]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[0])}")

prob = Problem()
prob.func_2()
print(f"\nProblem func_2 [-0.2,1.2] FR:")
print("\nAlgorithm output: ")
x_ = FR(np.array([-0.2,1.2]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[1])}")
print(f"\nProblem func_2 [-0.2,1.2] FR approximate gradient:")
print("\nAlgorithm output: ")
x_ = FR(np.array([-0.2,1.2]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[0])}")

prob = Problem()
prob.func_2()
print(f"\nProblem func_2:")
print("\nAlgorithm output SR1: ")
x_ = SR1(np.array([-0.2,1.2]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[0])}")
print(f"\nProblem func_2 [-0.2,1.2] SR1 approximate gradient:")
print("\nAlgorithm output: ")
x_ = SR1(np.array([-0.2,1.2]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[0])}")

**Function 2 [3.8,0.1] starting point**

In [ ]:
prob = Problem()
prob.func_2()
print(f"\nProblem func_2 [3.8,0.1] steepest descent:")
print("\nAlgorithm output: ")
x_ = steepest_descent(np.array([3.8,0.1]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[1])}")
print(f"\nProblem func_2 [3.8,0.1] steepest descent approximate gradient:")
print("\nAlgorithm output: ")
x_ = steepest_descent(np.array([3.8,0.1]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[1])}")


prob = Problem()
prob.func_2()
print(f"\nProblem func_2 [3.8,0.1] newton method:")
print("\nAlgorithm output: ")
x_ = newton_method(np.array([3.8,0.1]), prob.f, prob.grad_f, prob.hessian)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[1])}")
print(f"\nProblem func_2 [3.8,0.1] newton method approximate gradient:")
print("\nAlgorithm output: ")
x_ = newton_method(np.array([3.8,0.1]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[1])}")

prob = Problem()
prob.func_2()
print(f"\nProblem func_2 [3.8,0.1] FR:")
print("\nAlgorithm output: ")
x_ = FR(np.array([3.8,0.1]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[1])}")
print(f"\nProblem func_2 [3.8,0.1] FR approximate gradient:")
print("\nAlgorithm output: ")
x_ = FR(np.array([3.8,0.1]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[0])}")

prob = Problem()
prob.func_2()
print(f"\nProblem func_2:")
print("\nAlgorithm output SR1: ")
x_ = SR1(np.array([3.8,0.1]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[1])}")
print(f"\nProblem func_2 [3.8,0.1] SR1 approximate gradient:")
print("\nAlgorithm output: ")
x_ = SR1(np.array([3.8,0.1]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[1])}")

**Function 2 [0,0] starting point**

In [ ]:
prob = Problem()
prob.func_2()
print(f"\nProblem func_2 [0.0,0.0] steepest descent:")
print("\nAlgorithm output: ")
x_ = steepest_descent(np.array([0.0,0.0]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[1])}")
print(f"\nProblem func_2 [0.0,0.0] steepest descent approximate gradient:")
print("\nAlgorithm output: ")
x_ = steepest_descent(np.array([0.0,0.0]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[1])}")


prob = Problem()
prob.func_2()
#This raises linalgerror for singular matrix
"""print(f"\nProblem func_2 [0.0,0.0] newton method:")
print("\nAlgorithm output: ")
x_ = newton_method(np.array([0.0,0.0]), prob.f, prob.grad_f, prob.hessian)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[1])}")"""
print(f"\nProblem func_2 [0.0,0.0] newton method approximate gradient:")
print("\nAlgorithm output: ")
x_ = newton_method(np.array([0.0,0.0]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[0])}")

prob = Problem()
prob.func_2()
print(f"\nProblem func_2 [0.0,0.0] FR:")
print("\nAlgorithm output: ")
x_ = FR(np.array([0.0,0.0]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[0])}")
print(f"\nProblem func_2 [0.0,0.0] FR approximate gradient:")
print("\nAlgorithm output: ")
x_ = FR(np.array([0.0,0.0]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[0])}")

prob = Problem()
prob.func_2()
print(f"\nProblem func_2:")
print("\nAlgorithm output SR1: ")
x_ = SR1(np.array([0.0,0.0]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[0])}")
print(f"\nProblem func_2 [0.0,0.0] SR1 approximate gradient:")
print("\nAlgorithm output: ")
x_ = SR1(np.array([0.0,0.0]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[0])}")

**Function 2 [-1,0] starting point**

In [ ]:
prob = Problem()
prob.func_2()
print(f"\nProblem func_2 [-1,0] steepest descent:")
print("\nAlgorithm output: ")
x_ = steepest_descent(np.array([-1,0]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[0])}")
print(f"\nProblem func_2 [-1,0] steepest descent approximate gradient:")
print("\nAlgorithm output: ")
x_ = steepest_descent(np.array([-1,0]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[0])}")


prob = Problem()
prob.func_2()
print(f"\nProblem func_2 [-1,0] newton method:")
print("\nAlgorithm output: ")
x_ = newton_method(np.array([-1,0]), prob.f, prob.grad_f, prob.hessian)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[1])}")
print(f"\nProblem func_2 [-1,0] newton method approximate gradient:")
print("\nAlgorithm output: ")
x_ = newton_method(np.array([-1,0]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[0])}")

prob = Problem()
prob.func_2()
print(f"\nProblem func_2 [-1,0] FR:")
print("\nAlgorithm output: ")
x_ = FR(np.array([-1,0]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[0])}")
print(f"\nProblem func_2 [-1,0] FR approximate gradient:")
print("\nAlgorithm output: ")
x_ = FR(np.array([-1,0]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[0])}")

prob = Problem()
prob.func_2()
print(f"\nProblem func_2:")
print("\nAlgorithm output SR1: ")
x_ = SR1(np.array([-1,0]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[0])}")
print(f"\nProblem func_2 [-1,0] SR1 approximate gradient:")
print("\nAlgorithm output: ")
x_ = SR1(np.array([-1,0]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[0])}")

**Function 2 [0,-1] starting point**

In [ ]:
prob = Problem()
prob.func_2()
print(f"\nProblem func_2 [0.0,-1.0] steepest descent:")
print("\nAlgorithm output: ")
x_ = steepest_descent(np.array([0.0,-1.0]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[1])}")
print(f"\nProblem func_2 [0.0,-1.0] steepest descent approximate gradient:")
print("\nAlgorithm output: ")
x_ = steepest_descent(np.array([0.0,-1.0]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[1])}")


prob = Problem()
prob.func_2()
print(f"\nProblem func_2 [0.0,-1.0] newton method:")
print("\nAlgorithm output: ")
x_ = newton_method(np.array([0.0,-1.0]), prob.f, prob.grad_f, prob.hessian)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[0])}")
#this raises linalgerror singular matrix
"""print(f"\nProblem func_2 [0.0,-1.0] newton method approximate gradient:")
print("\nAlgorithm output: ")
x_ = newton_method(np.array([0.0,-1.0]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[0])}")"""

prob = Problem()
prob.func_2()
print(f"\nProblem func_2 [0.0,-1.0] FR:")
print("\nAlgorithm output: ")
x_ = FR(np.array([0.0,-1.0]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[0])}")
print(f"\nProblem func_2 [0.0,-1.0] FR approximate gradient:")
print("\nAlgorithm output: ")
x_ = FR(np.array([0.0,-1.0]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[1])}")

prob = Problem()
prob.func_2()
print(f"\nProblem func_2:")
print("\nAlgorithm output SR1: ")
x_ = SR1(np.array([0.0,-1.0]), prob.f, prob.grad_f)
print(f"\nactual minima: {prob.min_x}")
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[0])}")
print(f"\nProblem func_2 [0.0,-1.0] SR1 approximate gradient:")
print("\nAlgorithm output: ")
x_ = SR1(np.array([0.0,-1.0]), prob.f)
print(f"\nactual minima: {prob.min_x}")
grad = approx_grad(prob.f)
print(f"\nLast gradient norm: {norm(prob.grad_f(x_))}")
print(f"\nDifference to real solution: {norm(x_ - prob.min_x[0])}")